In [1]:
#for local
import sys
repo_root = "../"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

In [2]:
import torch
from language import DynamicLanguage, HELM, SMILES
from utils import HELMConverter

In [3]:
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling
from tokenizers import Tokenizer, models, pre_tokenizers, decoders
from tokenizers.processors import TemplateProcessing
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel
from transformers import GPT2Config
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

def train_gpt2_with_dynamiclanguage(lang: DynamicLanguage, dataset_path: str, training_args: TrainingArguments, test_size=0.1, block_size=None, additional_length=0, n_embd=128, n_layer=6, n_head=4):
    #additional_length: if block size is not defined, block size = max number of tokens in one sentence in the dataset + additional length

    #make dataset and build vocabs
    ds = load_dataset("text", data_files={"train": dataset_path})
    ds = ds["train"].train_test_split(test_size=test_size)
    lang.build_vocab(ds)

    ds_tokenized = ds.map(
        lambda x: {"input_ids": lang.sentence2ids(x["text"])},
        remove_columns=["text"], #remove text column
        batched=False
    )

    #set max length from dataset
    if (block_size == None):
        max_length_ds = max(
            max(len(x["input_ids"]) for x in ds_tokenized["train"]),
            max(len(x["input_ids"]) for x in ds_tokenized["test"])
        )
        block_size = max_length_ds + additional_length
        print("set max length to: " + str(block_size))

    token_bos = lang.bos_token()
    token_eos = lang.eos_token()
    token_pad = lang.pad_token()

    tok_model = models.WordLevel(vocab=lang._token2id)
    tok = Tokenizer(tok_model)
    tok.pre_tokenizer = pre_tokenizers.Sequence([])     #already done at DynamicLanguage.sentence2tokens
    tok.decoder            = decoders.Sequence([])
    tok.post_processor = TemplateProcessing(
        single=f"{token_bos} $0 {token_eos}",
        pair=f"{token_bos} $A {token_eos} $B:1 {token_eos}:1",
        special_tokens=[
            (token_bos, lang.bos_id()),
            (token_eos, lang.eos_id()),
        ],
    )

    hf_tokenizer = PreTrainedTokenizerFast(
        tokenizer_object=tok,
        bos_token=token_bos,
        eos_token=token_eos,
        pad_token=token_pad,
    )

    print("Is CUDA available: " + str(torch.cuda.is_available()))

    config = GPT2Config(
        vocab_size = len(lang.vocab()),
        n_positions = block_size,
        n_ctx = block_size,
        n_embd = n_embd,
        n_layer = n_layer,
        n_head = n_head,
        bos_token_id = lang.bos_id(),
        eos_token_id = lang.eos_id(),
        pad_token_id = lang.pad_id(),
    )

    model = GPT2LMHeadModel(config)
    print("num_params: " + str(model.num_parameters()))

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=hf_tokenizer,
        mlm=False, # mlm is set to false since this is for generation task
    )

    trainer = Trainer(
        model = model,
        args = training_args,
        train_dataset = ds_tokenized["train"],
        eval_dataset = ds_tokenized["test"],
        data_collator = data_collator,
    )

    trainer.train()

    return model, trainer

c:\Users\fsfsf\anaconda3\envs\molgen-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import yaml
import os
from utils import class_from_package

config_path = "config/train_smiles.yaml"
with open(os.path.join(repo_root, config_path)) as f:
    conf = yaml.safe_load(f)

output_dir = os.path.join(repo_root, conf.get("output_dir"))
lang_class = class_from_package("language", conf.get("lang_class"))
lang_conf = conf.get("lang_conf", {})
lang = lang_class(**lang_conf)
dataset_path = os.path.join(repo_root, conf.get("dataset_path"))

training_args = conf.get("training_args", {})
training_args["output_dir"] = output_dir
interval = conf.get("interval")
if interval == "epoch":
    training_args["eval_strategy"] = training_args["logging_strategy"] = training_args["save_strategy"] = "epoch"
if type(interval) == int:
    training_args["eval_strategy"] = "steps"
    training_args["eval_steps"] = training_args["logging_steps"] = training_args["save_steps"] = interval
    
training_args = TrainingArguments(**training_args)
test_size, n_embd, n_layer, n_head = (conf.get(k) for k in ("test_size", "n_embd", "n_layer", "n_head"))

model, trainer = train_gpt2_with_dynamiclanguage(lang=lang, dataset_path=dataset_path, training_args=training_args, test_size=test_size, n_embd=n_embd, n_layer=n_layer, n_head=n_head)

Map: 100%|██████████| 62364/62364 [00:02<00:00, 21846.55 examples/s]


set max length to: 74
Is CUDA available: True
num_params: 3195392


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
1000,1.641800,1.280252
2000,1.221100,1.107435
3000,1.114100,1.038580
4000,1.053800,0.985910
5000,1.017900,0.950774
6000,0.985500,0.923902
7000,0.964000,0.903528
8000,0.941300,0.889524
9000,0.926400,0.881803
10000,0.913800,0.861638


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [19]:
#save yaml and lang
import shutil
src = os.path.join(repo_root, config_path)
dst = os.path.join(output_dir, "setting.yaml")
shutil.copy(src, dst)

if lang.__class__.__name__ == "HELM":
    lib_files = [
        "chembl_35_monomer_library.xml",
        "chembl_35_monomer_library_diff.xml",
        "HELMCoreLibrary.json",
        "monomerLib2.0.json"
    ]
    lang.load_monomer_library(*[repo_root + f"data/helm/library/{name}" for name in lib_files], culling=True)
lang.save(os.path.join(output_dir, "smiles.lang"))

In [ ]:
#save model in colab
import shutil
from google.colab import files

dl_path = output_dir
shutil.make_archive(dl_path, 'zip', dl_path)
files.download(f'{dl_path}.zip')